# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.233207776581                   -0.50    8.0         
  2   -7.250272697540       -1.77       -1.41    1.0   8.13ms
  3   -7.251156925413       -3.05       -2.16    1.0   8.04ms
  4   -7.251210283179       -4.27       -2.09    2.0   10.1ms
  5   -7.251332265078       -3.91       -2.68    1.0   8.25ms
  6   -7.251337420759       -5.29       -3.02    2.0   9.85ms
  7   -7.251338659659       -5.91       -3.56    1.0   8.10ms
  8   -7.251338768828       -6.96       -3.81    2.0   10.1ms
  9   -7.251338792622       -7.62       -4.16    1.0   8.73ms
 10   -7.251338797963       -8.27       -4.69    2.0   9.76ms
 11   -7.251338798640       -9.17       -5.15    3.0   11.9ms
 12   -7.251338798698      -10.24       -5.68    1.0   8.12ms
 13   -7.251338798704      -11.23       -6.18    2.0   9.83ms
 14   -7.251338798704      -12.34       -6.34    2.0   11.1ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.04542619909873417
[ Info: Arnoldi iteration step 2: normres = 0.6329944695915305
[ Info: Arnoldi iteration step 3: normres = 0.8242360393581618
[ Info: Arnoldi iteration step 4: normres = 0.26277013771681196
[ Info: Arnoldi iteration step 5: normres = 0.515156749359443
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.77e-02, 8.12e-02, 4.56e-01, 2.23e-01, 1.80e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3185543654573989
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (7.36e-03, 1.46e-01, 2.29e-01, 1.34e-01, 5.81e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07652987547828187
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.76e-04, 1.18e-02, 1.39e-02, 4.67e-02, 5.23e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09580131286542369
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.15e-0